# Manual runs of VQC
### Note:
1. For the manual runs control values are set in modules.config
2. It also possible to run automatically

Import modules needed:

In [ ]:
import numpy as np
import copy
import time
from pathlib import Path
from time import strftime

from modules.helper_functions_tsp import (find_problem_size, 
                                          cost_fn_fact, 
                                          convert_bit_string_to_cycle, 
                                          update_parameters_using_gradient, 
                                          define_parameters, 
                                          create_initial_rotations,
                                          bind_weights, 
                                          vqc_circuit, 
                                          cost_func_evaluate, 
                                          find_run_stats,
                                          find_distances_array,
                                          detect_quantum_GPU_support, 
                                          calculate_hot_start_data
                                         )

from modules.graph_functions import (parameter_graph, 
                                     cost_graph_multi,
                                     )

from classes.MyDataLogger import MyDataLogger, MySubDataLogger

Set random seed so results are reproducible for testing.

In [ ]:
np.random.seed(42)

Import constants needed.  Note that all are read directly from modules.config:

In [ ]:
from modules.config import (ROTATIONS, 
                            SLICES, 
                            )

CHANGE_EACH_PARAMETER = True       # Iterate through each parameter in the circuit
PLOT_PARAMETER_EVALUATION = True   # Plot the evaluation of each parameter         

In [ ]:
if detect_quantum_GPU_support():
    print('Quantum GPU support detected')
else:
    print('Quantum GPU support not detected')

Instantiate data logger - this will create a folder for graphs and results:


In [ ]:
datalogger = MyDataLogger()
sdl = MySubDataLogger(runid = datalogger.runid)
sdl.update_general_constants_from_config()
sdl.update_quantum_constants_from_config()
sdl.validate_input()

From the number of locations visited find the qubits and longest binary string.

In [ ]:
sdl.qubits = find_problem_size(sdl)
print(f'There are {sdl.qubits} qubits needed for {sdl.locations} locations in the {sdl.formulation} formulation.')
print(f'Running mode {sdl.mode} with {sdl.shots} shots')

Data sources are held locally to avoid downstream dependencies.  Read the data, and print out the filename and best distance held in the data.  Read and validate the distance array.  This checks the array is the correct shape, and is symmetric.

In [ ]:
distance_array, best_dist = find_distances_array(sdl.locations, print_comments=True)

Define the VQC circuits with appropriate parameters, and draw the circuit.

In [ ]:
sdl.num_params = sdl.calculate_parameter_numbers()
print(f'Number of parameters to be optimized is {sdl.num_params}')
params = define_parameters(sdl)

print(f'Parameters are: {params}')
qc = vqc_circuit(sdl, params)

filename = Path.joinpath(datalogger.graph_sub_path, f'initial_unbound_circuit{sdl.mode}.pdf')
fig = qc.draw("mpl", style="clifford", filename=filename )

fig.savefig(filename, dpi=300, bbox_inches="tight")

qc.draw("mpl", style="clifford", filename=filename )

Determine the cost function.  This receives a binary list and returns a real value:

In [ ]:
cost_fn = cost_fn_fact(sdl, distance_array)



Iterate over the control parameters and do one experiment for each one of them.

Create random initial rotations, and bind these to the parameters in a new circuit `bc`.

In [ ]:
bin_hot_start_list=[]
if sdl.hot_start:
    bin_hot_start_list, hot_start_distance = calculate_hot_start_data(sdl, 
                                                                      distance_array,
                                                                      cost_fn,
                                                                      print_results=True,
                                                                     )
init_rots = create_initial_rotations(sdl, bin_hot_start_list,)
print(f'The initial parameters (weights) are {init_rots}')
bc = bind_weights(params, init_rots, qc)
filename = Path.joinpath(datalogger.graph_sub_path, f'initial_bound_circuit{sdl.mode}.pdf')
fig = bc.draw("mpl", style="clifford", filename=filename )

fig.savefig(filename, dpi=300, bbox_inches="tight")
bc.draw("mpl", style="clifford", filename=filename )

In [ ]:
print([p.name for p in qc.parameters])
print([str(p) for p in params])

Find average and lowest cost for the starting circuit, and print these:

In [ ]:
cost_start, lowest_to_date, _ = cost_func_evaluate(sdl, 
                                                   cost_fn, 
                                                   bc, 
                                                   average_slice=SLICES[0], 
                                                   )
print(f'For the starting circuit the average cost is {cost_start:.1f} and the lowest cost is {lowest_to_date:.1f}')

Next we test the impact of varying each parameter in turn on the average cost for the first slicing ratio mentioned.  These results are not saved with the data logger.

In [ ]:
if CHANGE_EACH_PARAMETER:
    average_slice = SLICES[0]
    param_values = np.array([(i * 2 * np.pi) / (ROTATIONS-1) for i in range(ROTATIONS)])
    cost_list_all, lowest_list_all, sliced_cost_list_all = [], [], []
    #need deep copy in case init_rots is changed
    for i in range(sdl.num_params):
        rots = copy.deepcopy(init_rots)
        cost_list, lowest_list, sliced_cost_list  = [], [], []
        for angle in param_values:
            rots[i] = angle
            bc = bind_weights(params, rots, qc)
            sliced_cost, _ , _ = cost_func_evaluate(sdl, 
                                                    cost_fn, 
                                                    bc, 
                                                    average_slice=average_slice, 
                                                    )
            cost, lowest, lowest_energy_bit_string = cost_func_evaluate(sdl,
                                                                        cost_fn, 
                                                                        bc, 
                                                                        average_slice=1, 
                                                                        )
            route_list = convert_bit_string_to_cycle(lowest_energy_bit_string, 
                                                     sdl.locations, 
                                                     gray=sdl.gray, 
                                                     method=sdl.formulation
                                                     )
            cost_list.append(cost)
            lowest_list.append(lowest)
            sliced_cost_list.append(sliced_cost)
        cost_list_all.append(cost_list)
        lowest_list_all.append(lowest_list)     
        sliced_cost_list_all.append(sliced_cost_list)
    items, hits, misses = cost_fn.report_cache_stats()
    cost_fn.clear_cache() #need to clear cache so statistics are not cumulative

The results of changing the parameters are printed out:

In [ ]:
if CHANGE_EACH_PARAMETER:
    filename = Path.joinpath(datalogger.graph_sub_path,'Parameter_values')
    main_title = f'Av and lowest energy (distance) found by changing each parameter in turn with slices = {SLICES[0]}'
    sub_title = 'Parameter '
    x_label = 'Gate rotation in Radians'
    param_names = [i for i in range(sdl.num_params)]
    cost_graph_multi(filename, 
                     param_names, 
                     param_values, 
                     cost_list_all, 
                     lowest_list_all, 
                     sliced_cost_list_all, 
                     best_dist, 
                     main_title, 
                     sub_title, 
                     x_label
                     )

The evolution of the parameters is calculated and stored:

In [ ]:
if PLOT_PARAMETER_EVALUATION:
    sdl.slice = SLICES[0]

    average_slice = SLICES[0]
    #only do for one slice

    output_data = update_parameters_using_gradient(sdl,
                                                   params=params,
                                                   rots=init_rots,
                                                   cost_fn=cost_fn,
                                                   qc=qc,
                                                  )

    index_l, sliced_l, lowest_l, gradient_l, average_l, parameter_l = output_data

    cost_fn.print_cache_stats
    cost_fn.clear_cache() #need to clear cache so statistics are not cumulative

The evolution of the parameters is plotted.

In [ ]:
if PLOT_PARAMETER_EVALUATION:
    legend = [params[i].name for i in range(len(params))] 
    short_title = 'Evolution of parameters -'
    full_title = short_title + f' {sdl.formulation} formulation, hot start = {sdl.hot_start}'
    filename = Path.joinpath(datalogger.graph_sub_path,short_title)
    parameter_graph(filename, full_title, index_l, parameter_l, legend)

Do a run for each slice, time, and save results.

In [ ]:
rots = copy.deepcopy(init_rots)
av_cost_list_all, lowest_list_all, sliced_cost_list_all = [], [], []
sdl_list = []
for i, slice in enumerate(SLICES):
    t0 = time.time()
    sdl_list.append(MySubDataLogger(runid=datalogger.runid))
    sdl_list[i].update_general_constants_from_config()
    sdl_list[i].update_quantum_constants_from_config()
    sdl_list[i].slice = slice
    sdl_list[i].qubits = sdl.qubits
    
    new_output_data = update_parameters_using_gradient(sdl_list[i],
                                                       params=params,
                                                       rots=init_rots,
                                                       cost_fn=cost_fn,
                                                       qc=qc,
                                                       )
                        
    sdl_list[i].index_list, sdl_list[i].sliced_list, sdl_list[i].lowest_list, _ , sdl_list[i].average_list, _ = new_output_data
    
    av_cost_list_all.append(sdl_list[i].average_list)
    lowest_list_all.append(sdl_list[i].lowest_list)
    sliced_cost_list_all.append(sdl_list[i].sliced_list)

    best_dist_found, iteration_found = find_run_stats(sdl_list[i].lowest_list)
    print(f'With a slice of {slice} the best distance found was {best_dist_found} at iteration {iteration_found }')
    print(f'This is compared to the best distance of {best_dist}, giving a quality of {best_dist/best_dist_found:.1%}')
    if sdl_list[i].hot_start:
        sdl_list[i].hot_start_dist = hot_start_distance
    sdl_list[i].best_dist_found = best_dist_found
    sdl_list[i].best_dist = best_dist
    sdl_list[i].iteration_found = iteration_found

    t1 = time.time()
    elapsed = t1-t0
    print(f'The time taken to run the code is {elapsed:.3f} seconds')
    sdl_list[i].elapsed = elapsed
    sdl_list[i].update_cache_statistics(cost_fn)
    sdl_list[i].save_results_to_csv()
    sdl_list[i].save_detailed_results()

In [ ]:
sdl_list[0].update_cache_statistics(cost_fn)

The results are saved.  Note, the formulation, hot_start, gradient type and locations are the same for each sub-run so are taken from the last run:

In [ ]:
short_title = 'Dist '
full_title = short_title + f' {sdl.formulation} formulation, hot start={sdl.hot_start}, '
full_title = full_title + f'gradient type={sdl.gradient_type}, eta={sdl.eta}, locs={sdl.locations}'
filename = Path.joinpath(datalogger.graph_sub_path,short_title)
sub_title = 'Results with slicing ratio '
x_label = 'Iterations'
if SLICES  == [1]:
    sliced_cost_list_all = None

cost_graph_multi(filename, 
                 SLICES, 
                 sdl_list[0].index_list,
                 av_cost_list_all, 
                 lowest_list_all, 
                 sliced_cost_list_all, 
                 best_dist, 
                 full_title, 
                 sub_title,
                 x_label,
                )